In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/src')

import numpy as np
from deformation import *
from scipy.optimize import minimize, fmin_l_bfgs_b, Bounds

torchdeviceId = torch.device('cuda:0') 
torchdtype = torch.float64

### Read and prepare data

In [ ]:
data = torch.load('../Data/amygdala.pt')
amyg_sour = data['Source']
amyg_tar = data['Target']

VS = torch.from_numpy(amyg_sour['x']).clone().to(dtype=torchdtype, device=torchdeviceId)
FS = torch.from_numpy(amyg_sour['G']).clone().to(dtype=torch.long, device=torchdeviceId)
VT = torch.from_numpy(amyg_tar['x']).clone().to(dtype=torchdtype, device=torchdeviceId)
FT = torch.from_numpy(amyg_tar['G']).clone().to(dtype=torch.long, device=torchdeviceId)

Source = mesh2var(VS,FS)
Target = mesh2var(VT,FT)

results = {'amyg_sour':amyg_sour, 'amyg_tar':amyg_tar}

### Set parameters

In [ ]:
'''model parameters'''
options = {}
options['defo_kernel_size'] = 4.75
options['odemethod'] = 'rk4'
options['nb_euler_steps'] = 15
options['kernel_geom'] = 'gaussian'
options['kernel_grass'] = 'binet'
options['sig_geom'] = 3
options['sig_grass'] = 1.1
options['weight_varifold_dist'] = 10
#options['weight_meta'] = .00001

'''optimization parameters'''
opt_par = {'gtol': 1e-6, 'disp': True, 'step_size':1e-4, 
           'maxiter': 200, 'maxls': 50}

### LDDMM-$L2$ model

In [ ]:
'''model parameters'''
options = {}
options['defo_kernel_size'] = 4.75
options['odemethod'] = 'rk4'
options['nb_euler_steps'] = 15
options['kernel_geom'] = 'gaussian'
options['kernel_grass'] = 'binet'
options['sig_geom'] = 3
options['sig_grass'] = 1.1
options['weight_varifold_dist'] = 10
options['weight_meta'] = 0
options['model'] = 'metapwm'

'''optimization parameters'''
opt_par = {'gtol': 1e-6, 'disp': True, 'step_size':1e-4, 
           'maxiter': 200, 'maxls': 50}


energy_scipy = enr_scipy(Source,Target,**options) #Energy function

p = np.zeros(list2vec(Source).shape[0])
a = .5*np.ones(Source[0].shape[0])
x0 = np.concatenate((p,a))

start = time.time()
#bounds = np.array([[-np.inf]*p.shape[0] + [1e-10]*a.shape[0], [np.inf]*(p.shape[0]+a.shape[0])]).T
#res = GD_adapted(energy_scipy,x0,bounds=bounds,**opt_par)
B =  np.array([[-np.inf]*p.shape[0] + [1e-10]*a.shape[0], [np.inf]*(p.shape[0]+a.shape[0])]).T
bounds = Bounds(B[:,0], B[:,1], keep_feasible=False)
res = minimize(energy_scipy, x0, method='L-BFGS-B', bounds=bounds, jac=True, options=opt_par) #Optimization

reg_time = round(time.time() - start, 2)

_,_, Enr_dic = energy_scipy(res.x)
results['metapwm'] = {'res':res,'Enr_dic':Enr_dic,'options':options,'time':reg_time} 

torch.save(results,'amygdala_L2_result.pt')